In [ ]:
import numpy as np # linear algebra
import pandas as pd 

In [ ]:
!unzip /content/drive/MyDrive/ARVhackathon/Data/train.zip -d /content/house-grade/

Archive:  /content/drive/MyDrive/ARVhackathon/Data/train.zip
  inflating: /content/house-grade/train/ChokChai4_img_13-7955799_100-6031141_a187-21598902886774_s270_y0_f90_1.jpg  
  inflating: /content/house-grade/train/ChokChai4_img_13-7956791_100-6031267_a187-21598902886774_s270_y0_f90_1.jpg  
  inflating: /content/house-grade/train/ChokChai4_img_13-7956791_100-6031267_a187-21598902886774_s90_y0_f90_1.jpg  
  inflating: /content/house-grade/train/ChokChai4_img_13-7961753_100-6031881_a185-978546002526_s90_y0_f90_1.jpg  
  inflating: /content/house-grade/train/ChokChai4_img_13-7969811_100-5906061_a180-5812875399312_s270_y0_f90_1.jpg  
  inflating: /content/house-grade/train/ChokChai4_img_13-7970811_100-5906071_a180-5812875399312_s90_y0_f90_1.jpg  
  inflating: /content/house-grade/train/ChokChai4_img_13-7971811_100-5906081_a180-5812875399312_s90_y0_f90_1.jpg  
  inflating: /content/house-grade/train/ChokChai4_img_13-7972811_100-5906092_a180-5812875399312_s90_y0_f90_1.jpg  
  inflating: /

In [ ]:
#Read meta data (ใช้รูป crop)
metadata = pd.read_csv('/content/drive/MyDrive/ARVhackathon/Data/train_with_augment.csv')

In [ ]:
metadata = metadata.sample(frac=1).reset_index(drop=True)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import BatchNormalization , Dense , Dropout , GlobalAveragePooling2D

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

DATA_PATH = '/content/house-grade/train/'

# use preprocessing_function match with model
image_generator = ImageDataGenerator(preprocessing_function = tf.keras.applications.efficientnet.preprocess_input)
train_generator = image_generator.flow_from_dataframe(metadata[:3000],# edit for metadata[:-10000] for real data
                                    directory = DATA_PATH,
                                    x_col ='image_name',
                                    class_mode = 'raw',
                                    y_col = 'class',
                                    batch_size = 32,
                                    seed = 69,
                                    shuffle = True,
                                    target_size=(300,300)
                                    )
validation_generator = image_generator.flow_from_dataframe(metadata[3000:],# edit for metadata[-10000:] for real data
                                    directory = DATA_PATH,
                                    x_col ='image_name',
                                    class_mode = 'raw',
                                    y_col = 'class',
                                    batch_size = 32,
                                    seed = 69,
                                    shuffle = True,
                                    target_size=(300,300)
                                    )

Found 3000 validated image filenames.
Found 634 validated image filenames.


In [ ]:
def createEffNetB3(input_shape):
    model = EfficientNetB3(
    include_top=False,
    weights="imagenet",
    input_shape=input_shape,)
    return model

In [ ]:
# declare backbone model
basemodel = createEffNetB3((300, 300, 3))
basemodel.trainable = True

In [ ]:
basemodel.summary()

Model: "efficientnetb3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 300, 300, 3)  0           ['input_2[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 300, 300, 3)  7          ['rescaling_1[0][0]']            
 )                                                                                                
                                                                                     

In [ ]:
def create_model(base_model):
    
    x =  BatchNormalization()(base_model.output)
    x =  GlobalAveragePooling2D()(x)
    
    # Font Family
    x = Dense(1024,activation = 'relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(1024,activation = 'relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(6,activation = 'softmax')(x)
 
    model = tf.keras.Model(inputs = base_model.input , outputs = output)
    
    return model

In [ ]:
model = create_model(basemodel)

In [ ]:
@tf.function
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [ ]:
lr = 1e-4 
epochs = 30
'''
model.compile(optimizer =  tf.keras.optimizers.Adam(learning_rate=lr),
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = ['acc']
            )
'''

model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=[macro_f1])

In [ ]:
from tensorflow.keras.callbacks import Callback , EarlyStopping , ModelCheckpoint , ReduceLROnPlateau

checkpoint = ModelCheckpoint( filepath = '/content/drive/MyDrive/ARVhackathon/model/Effnet/', monitor='val_loss', verbose=0, save_best_only=True,
                            save_weights_only=True, mode='auto', save_freq='epoch')
earlystop =  EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0,
                            mode='auto', baseline=None, restore_best_weights=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min')

callbacks = [checkpoint,reduce_lr,earlystop]

In [ ]:
history = model.fit(x=train_generator,
                    validation_data=validation_generator,
                    epochs=30,
                    callbacks=callbacks)

Epoch 1/30
94/94 [==============================] - 108s 900ms/step - loss: 1.0780 - macro_f1: 0.0781 - val_loss: 1.1895 - val_macro_f1: 0.1138 - lr: 1.0000e-04
Epoch 2/30
94/94 [==============================] - 81s 855ms/step - loss: 0.4580 - macro_f1: 0.1989 - val_loss: 0.5186 - val_macro_f1: 0.2018 - lr: 1.0000e-04
Epoch 3/30
94/94 [==============================] - 81s 852ms/step - loss: 0.2041 - macro_f1: 0.2244 - val_loss: 0.4726 - val_macro_f1: 0.2107 - lr: 1.0000e-04
Epoch 4/30
94/94 [==============================] - 81s 855ms/step - loss: 0.1236 - macro_f1: 0.2315 - val_loss: 0.4601 - val_macro_f1: 0.2187 - lr: 1.0000e-04
Epoch 5/30
94/94 [==============================] - 80s 848ms/step - loss: 0.0627 - macro_f1: 0.2337 - val_loss: 0.5777 - val_macro_f1: 0.2213 - lr: 1.0000e-04
Epoch 6/30
94/94 [==============================] - 80s 845ms/step - loss: 0.0622 - macro_f1: 0.2343 - val_loss: 0.5223 - val_macro_f1: 0.2249 - lr: 1.0000e-04
Epoch 7/30
94/94 [=====================